# Hybrid search with Qdrant

Vector search based on dense embeddings captures the semantics of the data, so you don't have to use the same terms in queries and documents to still be able to find the relevant items. However, historically we were also using some other methods which rely on the presence of the keywords. Methods such as Bag-of-Words, TFIDF and BM25 are still useful and in some cases should be preferred over the dense embeddings.

### Sparse vectors

Surprisingly, keyword-based search is also implemented as vector search, but these vectors are usually sparse. That means the majority of the dimensions of such a vector are just zeros. A non-zero value at a particular vector dimension indicates the presence of a term from the dictionary assigned to that position. In other words, in sparse vectors, we have a dictionary in which each word/phrase gets its unique position. Since vectors are sparse, the dictionary can theoretically grow indefinitely, as we can append a new term at the very end.

The fact of using a flexible dictionary, make the sparse vectors excel in exact matches, as they can cover texts that would be sets of random characters for the dense vectors - such as proper names or identifiers. Dense embedding models also have a dictionary, but once the model is trained, extending them is not that easy, and requires fine-tuning of the model. A typical user rarely goes that far.

### BM25

There are plenty of different options for creating sparse embeddings, but BM25 is an industry standard, and its most popular form comes from the 90s. It's a statistical model (no neural networks involved), which makes it really fast and lightweight. It's usually a solid baseline in search benchmarks so you should not ignore it.

BM25 stands for Best Matching 25, and it was just the 25th attempt to create a formula that calculates how relevant a particular document is, given a query. If you are interested in mathematical background, please check out the [Wikipedia page](https://en.wikipedia.org/wiki/Okapi_BM25) that describes it really well. In general, BM25 is a ranking function that helps search engines determine how relevant a document is to a query by combining two key concepts: `Term Frequency (TF) and Inverse Document Frequency (IDF)`.
1. The Term Frequency component rewards documents that contain the query terms multiple times, but with diminishing returns - so a document with 10 occurrences of a word isn't necessarily 10 times better than one with just 1 occurrence.
2. The Inverse Document Frequency part boosts the importance of rare words while reducing the weight of common words that appear in many documents, since rare terms are typically more informative for distinguishing relevant results.

BM25 also incorporates document length normalization to prevent longer documents from having an unfair advantage simply due to their size.

In our case, we'll use an implementation available in FastEmbed. Let's start with the basics.

---

## Step 0: Setup

Please refer to [sematic_search.ipynb](https://github.com/DataTalksClub/llm-zoomcamp/blob/826c47832692ecfcbf8df786de58ba12c8063982/02-vector-search/sematic_search.ipynb) notebook to set up the libraries required to interact with Qdrant and to create the embeddings. Similarly, please start Qdrant in a Docker container as described there, if it's not running on your machine yet.

If you skipped our previous lessons, the following commands will install all the packages and run Qdrant in a background container.

```sh
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

!docker run -d -p 6333:6333 -p 6334:6334 \
   -v "./qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant
```

## Step 1: Connect to Qdrant

Let's connect to Qdrant and test if that was successful by listing all the collections.

In [4]:
from qdrant_client import QdrantClient, models

client = QdrantClient('http://localhost:6333')
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-rag'), CollectionDescription(name='zoomcamp-faq')])

## Step 2: Sparse vector search with BM25

We are going to use the same dataset as before. Let's download it and load into Qdrant, but this time we are going to create sparse vectors with BM25 only.

In [3]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [5]:
# Create the collection with specified sparse vector parameters
collection_name = 'zoomcamp-sparse'
client.create_collection(
    collection_name=collection_name,
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

True

FastEmbed comes with a BM25 implementation that we can use as any other model.


In [6]:
import uuid

# send the points to the collection
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc['text'],
                    model='Qdrant/bm25'
                )
            },
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        for course in documents_raw
        for doc in course['documents']
    ]
)

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 58.62it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

You might be surprised how fast the upload operation was. BM25 does not require a neural network, so it is fast compared to dense embedding models.

---

## Step 3: Running sparse vector search with BM25

Right now, our vectors are ready to be searched over. Let's create a helper function.

In [7]:
def search (query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model="Qdrant/bm25"
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )
    return results.points

In [8]:
results = search(query="Qdrant")
results

[]

Sparse vectors can return no results, if none of the keywords from the query were ever used in the documents. No matter if there are some synonyms. Terminology does matter.

In [9]:
results = search(query='pandas')
print(results[0].payload['text'])

You can use round() function or f-strings
round(number, 4)  - this will round number up to 4 decimal places
print(f'Average mark for the Homework is {avg:.3f}') - using F string
Also there is pandas.Series. round idf you need to round values in the whole Series
Please check the documentation
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.round.html#pandas.Series.round
Added by Olga Rudakova


Scores returned by BM25 are not calculated with cosine similarity, but with BM25 formula. They are not bounded to a specific range, but are virtually unbounded. Let's see how they may look like.

In [10]:
results[0].score

6.0392046

That's an important observation before we start implementing hybrid search.

### Natural language like queries

Let's try again with a random question from our dataset to see how well sparse vector search can work with longer, natural language like queries.

In [11]:
import random
import json

random.seed(202506)

course = random.choice(documents_raw)
course_piece = random.choice(course["documents"])
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [12]:
results = search(course_piece["question"])
print(results[0].payload["text"])

The trial dbt account provides access to dbt API. Job will still be needed to be added manually. Airflow will run the job using a python operator calling the API. You will need to provide api key, job id, etc. (be careful not committing it to Github).
Detailed explanation here: https://docs.getdbt.com/blog/dbt-airflow-spiritual-alignment
Source code example here: https://github.com/sungchun12/airflow-toolkit/blob/95d40ac76122de337e1b1cdc8eed35ba1c3051ed/dags/examples/dbt_cloud_example.py


## Step 4: Qdrant Universal Query API - prefetching

Qdrant's `.query_points` method allows building multi-step search pipelines which can incorporate various methods into a single call. For example, we can retrieve some candidates with dense vector search, and then rerank them with sparse search, or use a fast method for initial retrieval and precise, but slow, reranking.

```
ascii
┌─────────────┐           ┌─────────────┐
│             │           │             │
│  Retrieval  │ ────────► │  Reranking  │
│             │           │             │
└─────────────┘           └─────────────┘
```

Let's create another collection that will keep both dense and sparse representations. Qdrant named vectors allow us to store multiple representations per point and it proves useful especially when we want to use mulitple models in our applications.

In [13]:
# create the collection with both vector types
collection_name_new = 'zoomcamp-sparse-and-dense'
client.create_collection(
    collection_name=collection_name_new,
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE
        )
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

True

We have to upload all the vectors into the newly created collection.



In [14]:
client.upsert(
    collection_name=collection_name_new,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc['text'],
                    model="jinaai/jina-embeddings-v2-small-en"
                ),
                "bm25": models.Document(
                    text=doc['text'],
                    model="Qdrant/bm25"
                )
            },
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        for course in documents_raw
        for doc in course['documents']
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
def multi_stage_search (query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name_new,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en"
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            )
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25"
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )
    return results.points

In [16]:
print(json.dumps(course_piece, indent=2))

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}


In [17]:
results = multi_stage_search(course_piece["question"])
print(results[0].payload["text"])

Problem description. How can we connect s3 bucket to MLFLOW?
Solution: Use boto3 and AWS CLI to store access keys. The access keys are what will be used by boto3 (AWS' Python API tool) to connect with the AWS servers. If there are no Access Keys how can they make sure that they have the right to access this Bucket? Maybe you're a malicious actor (Hacker for ex). The keys must be present for boto3 to talk to the AWS servers and they will provide access to the Bucket if you possess the right permissions. You can always set the Bucket as public so anyone can access it, now you don't need access keys because AWS won't care.
Read more here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html
Added by Akshit Miglani


## Step 5: Building Hybrid Search

In real production systems, you don't need to choose just one vector type. You never know what kind of queries your users will send to the system. E-commerce search might be just fine with lexical search on top of sparse vectors, as people will tend to send keywords, but in conversational systems, such as chatbots, natural language questions might be more frequent. Using one model as a retriever and another one as reranker is not the only way of how to use dense and sparse in a single system.

Hybrid Search is a technique for combining results coming from different search methods - for example dense and sparse. There isn't a clear definition of how exactly to implement it, as the main problem is how to mix results coming from methods which are incompatible. Dense and sparse search scores can't be compared directly, so we need another method that will order the final results somehow.

There are two terms important for Hybrid Search: `fusion` and `reranking`.

### Fusion

Fusion is a set of methods which work on the scores/ranking as returned by the individual methods. There are various ways of how to achieve that, but Reciprocal Rank Fusion is the most popular technique. It is based on the rankings of the documents in each methods used, and these rankings are used to calculate the final scores. You will never calculate these scores, as Qdrant has some built-in capabilities that we will use. However, the following example can give you a rough intuition:

|Document|Dense ranking|Sparse ranking|RRF score|Final ranking|
|---|---|---|---|---|
|D1|1|5|0.0318|2|
|D2|2|4|0.0317|3|
|D3|3|2|0.0320|1|
|D4|4|3|0.0315|5|
|D5|5|1|0.0318|2|

In [18]:
def rrf_search (query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name_new,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en"
                ),
                using="jina-small",
                limit=(5 * limit)
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25"
                ),
                using="bm25",
                limit=(5 * limit)
            )
        ],
        # fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True
    )

    return results.points

In [19]:
results = rrf_search(course_piece["question"])
print(json.dumps(course_piece, indent=2))
print(results[0].payload["text"])

{
  "text": "Even though the upload works using aws cli and boto3 in Jupyter notebook.\nSolution set the AWS_PROFILE environment variable (the default profile is called default)",
  "section": "Module 4: Deployment",
  "question": "Uploading to s3 fails with An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.\""
}
The trial dbt account provides access to dbt API. Job will still be needed to be added manually. Airflow will run the job using a python operator calling the API. You will need to provide api key, job id, etc. (be careful not committing it to Github).
Detailed explanation here: https://docs.getdbt.com/blog/dbt-airflow-spiritual-alignment
Source code example here: https://github.com/sungchun12/airflow-toolkit/blob/95d40ac76122de337e1b1cdc8eed35ba1c3051ed/dags/examples/dbt_cloud_example.py


### Reranking

Reranking is a broader term related to Hybrid Search. Fusion is one of the ways to rerank the results of multiple methods, but you can also apply a slower method that won't be effective enough to search over all the documents. But there is more to it. Business rules are often important for retrieval, as you prefer to show documents coming from the most recent news, for instance.

## Next steps

Dense and sparse vector search methods might not be enough in some cases, but both are fast enough to be used as initial retrievers. Plenty of more accurate yet slower methods exist, such as cross-encoders or [multivector representations](https://qdrant.tech/documentation/advanced-tutorials/using-multivector-representations/). These topics are definitely more advanced, and we won't cover them right now. However, it's good to mention them so you are aware they exist.